# Scraping Data

In [2]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import requests

In [ ]:
original_url = "https://www5.himovies.to/movie"
response = requests.get(original_url)
print(response)
soup = BeautifulSoup(response.content, 'html.parser')
print(original_url)

In [ ]:
page_jumps=0
a=list()
for i in range(1092):
    temp_url=original_url
    page_jumps+=1
    temp_url+="?page="+str(page_jumps)
    print(temp_url)
    response = requests.get(temp_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = [a['href'] for a in soup.select('a[href]')]
    
    for line in links:
        if '/movie/' in line:
            a.append(line)
            
links_df = pd.DataFrame({"names":a})
links_df=links_df.drop_duplicates()
links_df

In [ ]:
links_df.to_csv(r'C:\Users\almog\Downloads\data science project python\data\links_df.csv', index=False)

In [ ]:
links_df=pd.read_csv('data\links_df.csv')

In [ ]:
check_df = pd.read_csv('data\links_df.csv')
links_df_updated=check_df.copy()
links_df_updated

In [ ]:
update_url = "https://www5.himovies.to"
i=0
while(i<links_df.shape[0]):
    temp=update_url
    links_df_updated.iloc[i,0]=temp+links_df_updated.iloc[i,0]
    i+=1
links_df_updated

In [ ]:
links_df_updated.to_csv(r'C:\Users\almog\Downloads\data science project python\data\links_df_updated.csv', index=False)

In [ ]:
movie_name=list()
release_year=list()
imdb_grades=list()
country=list()
genres=list()
duration=list()
production=list()
casts=list()

In [ ]:
links_df_updated.shape[0]

In [ ]:
i=0
while(i<links_df_updated.shape[0]):
    response = requests.get(links_df_updated.iloc[i,0])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    ## taking the movie name
    soup_h2_href=soup.find_all("h2",attrs={"href":""})
    for check_row in soup_h2_href:
        data=str(check_row.text)
        movie_name.append(data)
        break
    
    ### taking the movie grade by imdb:
    
    soup_button_btn=soup.find_all("button",class_="btn btn-sm btn-radius btn-warning btn-imdb")
    for check_row in soup_button_btn:
        data=str(check_row.text)
        temp_data=data.split(sep='IMDB: ')
        clean_data=temp_data[1]
        if(clean_data!='N/A'):
            clean_data=float(clean_data)
            imdb_grades.append(clean_data)
        else:
            imdb_grades.append(np.nan)
    
    ### taking the released, genre, casts
    
    soup_div_xl5=soup.find_all("div",class_="col-xl-5 col-lg-6 col-md-8 col-sm-12")
    for check_row in soup_div_xl5:
        data=check_row.find_all(strong_="",string='')
        flag1=flag2=flag3=True
        for row in data:
            if("Released:  " in row.text and flag1==True):
                flag1=False
                s1=row.text.strip()
                s1=s1.replace("Released:","")
                s1=s1.replace(" ","")
                if(s1!='N/A'):
                    release_year.append(s1)
                else:
                    release_year.append(np.nan)
                
            if("Genre: " in row.text and flag2==True):
                flag2=False
                s2=row.text.strip()
                s2=s2.replace("Genre:","")
                s2=s2.replace("\n","")
                s2=s2.replace(" ","")
                if(s2!='N/A'):
                    genres.append(s2)
                else:
                    genres.append(np.nan)
                
            if("Casts: " in row.text and flag3==True):
                flag3=False
                s3=row.text.strip()
                s3=s3.replace("Casts:","")
                s3=s3.replace("\n","")
                s3=s3.replace(" ","")
                if(s3!='N/A'):
                    casts.append(s3)
                else:
                    casts.append(np.nan)
                
                
    soup_div_xl6=soup.find_all("div",class_="col-xl-6 col-lg-6 col-md-4 col-sm-12")
    for check_row in soup_div_xl6:
        data=check_row.find_all(strong_="",string='')
        
        flag1=flag2=flag3=True
        for row in data:
            if("Duration:" in row.text and flag1==True):
                flag1=False
                s1=row.text.strip()
                s1=s1.replace("Duration:","")
                s1=s1.replace("\n","")
                s1=s1.replace(" ","")
                s1=s1.replace("min","")
                if(s1!='N/A'):
                    s1=int(s1)
                    duration.append(s1)
                else:
                    duration.append(np.nan)
                
            if("Country:" in row.text and flag2==True):
                flag2=False
                s2=row.text.strip()
                s2=s2.replace("Country:","")
                s2=s2.replace("\n","")
                s2=s2.replace(" ","")
                if(s2!='N/A'):
                    country.append(s2)
                else:
                    country.append(np.nan)
                
            if("Production:" in row.text and flag3==True):
                flag3=False
                s3=row.text.strip()
                s3=s3.replace("Production:","")
                s3=s3.replace("\n","")
                s3=s3.replace(" ","")
                if(s3!='N/A'):
                    production.append(s3)
                else:
                    production.append(np.nan)
    print(i)
    i+=1

In [ ]:
himovies_df= pd.DataFrame({"movie_name":movie_name,"release_year":release_year,
                          "imdb_grades":imdb_grades,"genres":genres,
                          "casts":casts,"duration in min":duration,"country":country,
                          "production":production})

In [9]:
himovies_df.rename(columns = {'movie_name':'Movie_Name', 'release_year':'Release_Year','imdb_grades':'IMDB_Score',
                             'genres':'Genres','casts':'Casts','duration in min':'Duration','country':'Country',
                             'production':'Production'}, inplace = True)

In [11]:
#himovies_df.to_csv(r'C:\Users\almog\Downloads\data science project python\data\ALL_RAW_DATA.csv', index=False)
himovies_df.to_csv(r'C:\Users\kon31\OneDrive\Desktop\data science project python\data\ALL_RAW_DATA.csv', index=False)